In [ ]:
# show matplotlib plots in-line
%matplotlib notebook

# allows for automatic reloading of imports and makes it unncessecary to restart the kernel
# whenever a function is changed
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt

# to be able to import scripts from scripts folder even though it's not a child repository
import sys
sys.path.append('..//scripts//')

from classes.Settings import Settings
from classes.Radii import Radii

from calc_avg_fragment import calc_avg_frag

from helpers.density_helpers import count_bins_in_vdw

In [ ]:
filenames = []
filenames.append("..\\data\\RNO2\\RNO2_CF_vdw.5\\RNO2_CF_vdw.5.csv")
filenames.append("..\\data\\RCOMe\\RCOMe_R2CO_vdw.5\\RCOMe_R2CO_vdw.5.csv")
filenames.append("..\\data\\RC6H5\\RC6H5_CF_vdw.5\\RC6H5_CF_vdw.5.csv")
filenames.append("..\\data\\RC6F5\\RC6F5_CF_vdw.5\\RC6F5_CF_vdw.5.csv")

avg_fragments = []
for filename in filenames:
    settings = Settings("..\\..\\", filename)

    radii = Radii(settings.get_radii_csv_name())
    
    df = pd.read_csv(settings.get_aligned_csv_filename())
    avg_fragment = calc_avg_frag(df, settings, radii)
    avg_fragments.append(avg_fragment)

In [ ]:
names = ['RNO2', 'RCOMe', 'RC6H5', 'RC6F5']

for name, avg_fragment in zip(names, avg_fragments):
    fig, ax = plt.subplots(figsize=(4,4))
    plt.title(name)
    
    for _, atom in avg_fragment.iterrows():
        linewidth = .5
        # plot the (average of the) central group
        if 'R' in atom.label:
            linewidth = 2.5
            color = 'green'
            label = 'R'
        elif 'H' in atom.symbol:
            color = 'grey'
            label = 'H'
        elif 'O' in atom.symbol:
            color='red'
            label = 'O'
        elif 'F' in atom.symbol:
            color='orchid'
            label = 'F'
        elif 'N' in atom.symbol:
            color='blue'
            label = 'N'
        elif 'C' in atom.symbol:
            color = 'black'
            label ='C'            
        else:
            color = 'cyan'
            label ='other?'
            
        plt.scatter(atom.x, atom.y, color=color, label=label)

        draw_circle = plt.Circle((atom.x, atom.y), atom.vdw_radius, fill=False, color=color, 
                                  linewidth=linewidth*0.5, linestyle=':')

        ax.set_aspect(1)
        ax.add_artist(draw_circle)

        draw_circle = plt.Circle((atom.x, atom.y), atom.vdw_radius + 0.5 + 1.09, fill=False, 
                                  color=color, linewidth=linewidth)

        ax.set_aspect(1)
        ax.add_artist(draw_circle)

    plt.xlabel('X coordinate')
    plt.ylabel('Y coordinate')
    
    plt.xlim(-6,8)
    plt.ylim(-7,7)
    
    handles, labels = ax.get_legend_handles_labels()
    unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
    plt.legend(*zip(*unique))

    plt.savefig('../../results/plots/overlap_R_volumes_' + name + '.png')
    plt.show()

In [ ]:
# resolution for volume calculation is 0.1
resolution = 0.1

for name, avg_fragment in zip(names, avg_fragments):
    print('\n' + name)
    tolerance = 0.5
    vdw_contact = 1.09
    extra = tolerance + vdw_contact
    
    avg_fragment["label"] = avg_fragment["label"].str.upper()
    avg_fragment_without_R = avg_fragment[~avg_fragment.label.str.contains("R")].copy()
    only_R = avg_fragment[avg_fragment.label.str.contains("R")].copy()
    
    volume = count_bins_in_vdw(avg_fragment=avg_fragment, extra=extra, resolution=resolution)
    print('with R (total)                :', round(volume, 1))
    
    # with half R 
    volume_max = count_bins_in_vdw(avg_fragment=avg_fragment, extra=extra, resolution=resolution)
    volume_central = count_bins_in_vdw(avg_fragment=avg_fragment, extra=0, resolution=resolution)
    
    volume_R_min = count_bins_in_vdw(avg_fragment=only_R, extra=0, resolution=resolution)
    volume_R_max = count_bins_in_vdw(avg_fragment=only_R, extra=extra, resolution=resolution)
    
    volume2 = (volume_max - volume_R_max/2) - (volume_central - volume_R_min/2)
    percent1 = (volume2 - volume) / volume * 100

    print('with half R                   :', round(volume2, 1), '    ', round(abs(percent1), 2), '% difference')
    
    # with R in min, not in max 
    volume_max = count_bins_in_vdw(avg_fragment=avg_fragment_without_R, extra=extra, resolution=resolution)
    volume_central = count_bins_in_vdw(avg_fragment=avg_fragment, extra=0, resolution=resolution)
    
    volume3 = volume_max - volume_central
    percent2 = (volume3 - volume) / volume * 100

    print('with R in min, not in max     :', round(volume3, 1), '    ', round(abs(percent2), 2), '% difference')
    
    
    # half R inmin, not in max:
    volume_max = count_bins_in_vdw(avg_fragment=avg_fragment_without_R, extra=extra, resolution=resolution)
    volume_central = count_bins_in_vdw(avg_fragment=avg_fragment, extra=0, resolution=resolution)
    
    volume_R_min = count_bins_in_vdw(avg_fragment=only_R, extra=0, resolution=resolution)
    
    volume4 = (volume_max) - (volume_central - volume_R_min/2)
    percent3 = (volume4 - volume) / volume * 100

    print('with half R in min, not in max:', round(volume4, 1), '    ', round(abs(percent3), 2), '% difference')    